# In this notebook you will find an example of how wer found the hyperparameters used for our models and the logic we have followed:
The only difference with the notebook model_test.ipynb is that in this case instead of droping the columns of the readings we will drop the columns that have nan readings. Then we will compare the final results and see what is the best model to do predictions and use for production

In [21]:
import pandas as pd
import numpy as np
import pickle
from test_evaluate_functions.test_evaluate_functions import calculate_accuracies, test_model, variable_importance
from training_functions.training_functions import calculate_binary_variables, split, split_x_y, evaluate_models, train_ada_boost, split_easy_and_sudden_errors

# Drop nan rows:

In [22]:
dff = pd.read_csv("/home/leibniz/Desktop/IHMAN/preprocessing_results/out_lights.csv")
print("Reading done!")
# drop nan rows:
df = dff.loc[~dff["ActivePeak_current_week"].isna()]

# Drop some usless columns for the model:
drop_cols = [
    col for col in df.columns if
        (col in ["lat", "lon"]) | 
        (col == "Unnamed: 0") |
        (col.startswith("week")) |
        (col == "type") |
        (col == "ebox_id") |
        (col == "location") |
        (col == "id")
]
df = df.drop(drop_cols, axis=1)

# Interpolate some left missing values:
df = df.fillna(df.mean(numeric_only=True))

Reading done!


In [23]:
df = calculate_binary_variables(df)
# Preprocessing and split:
df["current_week"] = pd.to_datetime(df["current_week"])
train, validation, test = split(df, n_weeks=77)

cols_to_train = df.drop(["current_week", "hours_next_four_weeks", "error_next_four_weeks", "hours_week+1", "hours_week+2", "hours_week+3", "hours_week+4"], axis=1).columns
x_train, y_train = split_x_y(train, cols_to_train)
x_validation, y_validation = split_x_y(validation, cols_to_train)
x_test, y_test = split_x_y(test, cols_to_train)

0.7606851470373555
0.32915929993025256
0.38210248008953623
0.2887382199802112
2017-03-20 00:00:00
2020-03-16 00:00:00
2020-03-23 00:00:00
2021-09-06 00:00:00
2021-09-13 00:00:00
2023-03-06 00:00:00


### Train model with the whole train dataset:

In [8]:
model_results = evaluate_models(
    max_depth_l = [1, 2, 3],
    n_estimators_l = [1, 2, 5, 10, 15, 20, 30],
    lr_l = [0.1, 0.2, 0.5],
    prob_threshold_l = np.arange(0.45, 0.55, 0.02).tolist(),
    x_train = x_train,
    y_train = y_train,
    x_validation = x_validation,
    y_validation = y_validation
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_results = model_results.append(row_to_append, ignore_index=True)
/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarnin

KeyboardInterrupt: 

In [10]:
model_results.loc[(model_results["yes_accuracy"] >= model_results["yes_accuracy"].max()-0.05)]

,learning_rate,n_estimators,total_accuracy,yes_accuracy,no_accuracy,prob_threshold,tree_depth
66,0.5,10.0,0.969606,0.906482,0.990558,0.45,1.0
84,0.5,15.0,0.966889,0.909463,0.985950,0.45,1.0
102,0.5,20.0,0.965997,0.917384,0.982133,0.45,1.0
103,0.5,20.0,0.969245,0.908526,0.989399,0.47,1.0
120,0.5,30.0,0.662818,0.954774,0.565912,0.45,1.0
121,0.5,30.0,0.967313,0.915084,0.984649,0.47,1.0
246,0.5,30.0,0.787282,0.915510,0.744721,0.45,2.0
372,0.5,30.0,0.820181,0.934759,0.782150,0.45,3.0
373,0.5,30.0,0.957253,0.905460,0.974444,0.47,3.0


We are going with the model lr=0.5, n_estimators=30, prob=0.47, depth=1

In [24]:
ada_model = train_ada_boost(
    max_depth_tree = 1,
    n_estimators = 30,
    lr = 0.5,
    x  = x_train,
    y = y_train
)
# Test:
test_model(
    model = ada_model,
    x = x_test, 
    y = y_test,
    prob_threshold = 0.47
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'total_accuracy': 0.9632043143643616,
 'yes_accuracy': 0.91628145865434,
 'no_accuracy': 0.9808636486643213}

In [25]:
# Check of the importance of the variables:
feature_importances = variable_importance(
    ada_model = ada_model, 
    trained_columns = cols_to_train
)
feature_importances.loc[feature_importances["importance"] != 0]

/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leib

,variable,importance
0,hours_week-4,0.033333
2,hours_week-2,0.033333
3,hours_week-1,0.033333
4,hours_current_week,0.133333
8,ReactivePeak_week-1,0.033333
24,powerReactive_p2_week-1,0.033333
28,powerReactive_p2_current_week,0.033333
29,powerReactive_p3_current_week,0.066667
30,ActivePeak_week-4,0.033333
38,powerActive_p3_week-4,0.033333


We can see that in this case the model does use the readings for doing predictions

### Sudden errors
We have a suspicion that the model we just trained does a very simple classification task using the errors of the last weeks, If the model finds errors in that weeks then it predicts an error in the following 4 weeks because it is normal to have errors recurrent errors.

From now on we will denote this errors as **"easy errors"**. This kind of error is easy to predict and we are more interested in predicting the from now on denoted **"sudden errors"**. This kind of errors are errors that appear without having a hard history of errors in the past weeks.

For example an error of type sudden whould be a lampost that suffers and error but we have all the variables hors_week-i at 0 or even one of them >0.

We will define an error as sudden error if we have 2 or less past weeks with errors. We consider the variable current_week to be a past week.

In [26]:
test_sudden, test_easy = split_easy_and_sudden_errors(test)

x_test_sudden, y_test_sudden = split_x_y(test_sudden, cols_to_train)
x_test_easy, y_test_easy = split_x_y(test_easy, cols_to_train)

Let's apply the model "ada_model" to the splited test dataset to verify our theory:

In [27]:
test_model(ada_model, x_test_sudden, y_test_sudden, 0.47)

{'total_accuracy': 0.2453748782862707,
 'yes_accuracy': 0.2453748782862707,
 'no_accuracy': None}

In [28]:
test_model(ada_model, x_test_easy, y_test_easy, 0.47)

{'total_accuracy': 0.984526391901663,
 'yes_accuracy': 0.9954065227377125,
 'no_accuracy': 0.9808636486643213}

We can see that the model almost has a perfect fit with the test_easy dataset. On the other hand, it has very bad performance on the test_sudden dataset. We can not consider ada_model to be the optimal predictor because it has a limited capacity of detecting sudden errors. We will have to design a better model for this cases.

To do it, the best way will be to split the datasets train, validation and test to train the sub-model so we make sure that the models are trained with the same datasests or subdatasets.

In [29]:
train_sudden, train_easy = split_easy_and_sudden_errors(train)
validation_sudden, validation_easy = split_easy_and_sudden_errors(validation)

# The dataset train_sudden has just rows with the column "error_next_four_weeks" = "Yes" so we will have to add
# some "No" rows in order to the model to train. To do this we will simply add a random sample of length len(train_sudden)
# of "No" rows to the dataset train_sudden.
train_sudden = pd.concat(
    [
        train.loc[train["error_next_four_weeks"] == "No"].sample(len(train_sudden)),
        train_sudden
    ],
    sort=True
).sample(frac=1.0, random_state=42)

validation_sudden = pd.concat(
    [
        validation.loc[validation["error_next_four_weeks"] == "No"].sample(len(validation_sudden)),
        validation_sudden
    ],
    sort=True
).sample(frac=1.0, random_state=42)

x_train_sudden, y_train_sudden = split_x_y(train_sudden, cols_to_train)
x_validation_sudden, y_validation_sudden = split_x_y(validation_sudden, cols_to_train)

In [19]:
# Train the model:
model_sudden_results = evaluate_models(
    max_depth_l = [1, 2, 3],
    n_estimators_l = [1, 10, 30, 100],
    lr_l = [0.1, 0.2, 0.5],
    prob_threshold_l = np.arange(0.45, 0.55, 0.02).tolist(),
    x_train = x_train_sudden,
    y_train = y_train_sudden,
    x_validation = x_validation_sudden,
    y_validation = y_validation_sudden
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_results = model_results.append(row_to_append, ignore_index=True)
/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarnin

KeyboardInterrupt: 

In [20]:
model_sudden_results.loc[(model_sudden_results["no_accuracy"] >= model_sudden_results["no_accuracy"].max()-0.37) & (model_sudden_results["yes_accuracy"] >= model_sudden_results["yes_accuracy"].max()-0.37)]

NameError: name 'model_sudden_results' is not defined

We will go with the model max_depth = 3, n_estimators=100, lr=0.2, prob=0.49

In [30]:
ada_sudden_model = train_ada_boost(
    max_depth_tree = 3,
    n_estimators = 100,
    lr = 0.2,
    x  = x_train_sudden,
    y = y_train_sudden
)
# Test:
test_model(
    model = ada_sudden_model, 
    x = x_test_sudden, 
    y = y_test_sudden,
    prob_threshold = 0.49
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'total_accuracy': 0.4936708860759494,
 'yes_accuracy': 0.4936708860759494,
 'no_accuracy': None}

We are not getting much better results than when dropping the readings columns. Since this model is based in much more less data and more columns, there is no need to develop it further with the adboc aproach.

# Export the best model:

In [31]:
with open("predictive_models/ada_model_readings.pk1", "wb") as file:
    pickle.dump(ada_model, file)

# Save the probability thresholds:
with open("predictive_models/ada_prob_readings.pk1", "wb") as file:
    pickle.dump({"prob_ada_model": 0.47}, file)